In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas

#import make_model_count as mm

#const variables 
col_recall_number = "RecallCount"
col_date = "Date"
col_month = "Month"

def get_df():
    # get_df function load the csv file and creates the panda dataframe and return it
    # Furthermore, it also change the names of the columns in order to make these names are more readable
    vehicle_df = pd.read_csv("./resources/vrdb_60days_daily.csv")
    vehicle_df.head()
    vehicle_df = vehicle_df[["RECALL_NUMBER_NUM", "RECALL_DATE_DTE"]]
    vehicle_df.columns = [col_recall_number, col_date]
    vehicle_df = vehicle_df.groupby([col_date]).count()
    vehicle_df = vehicle_df.sort_values(by=col_date)
    vehicle_df = vehicle_df.reset_index()
    vehicle_df = vehicle_df[vehicle_df[col_date].str.contains("2022")]
    vehicle_df[col_month] = vehicle_df[col_date].astype(str).str[5:7]
    vehicle_df = vehicle_df.groupby([col_month]).sum()

    return vehicle_df

def recalls_temperature():
    vehicle_df = get_df()
    return vehicle_df

recalls_temperature()


# "RECALL_NUMBER_NUM",
# "YEAR",
# "MANUFACTURER_RECALL_NO_TXT",
# "CATEGORY_ETXT",
# "CATEGORY_FTXT",
# "MAKE_NAME_NM",
# "MODEL_NAME_NM",
# "UNIT_AFFECTED_NBR",
# "SYSTEM_TYPE_ETXT",
# "SYSTEM_TYPE_FTXT",
# "NOTIFICATION_TYPE_ETXT",
# "NOTIFICATION_TYPE_FTXT",
# "COMMENT_ETXT",
# "COMMENT_FTXT",
# "RECALL_DATE_DTE",
# "LAST_UPDATE_DATE",
# "MANUFACTURER_NAME_NM"

,RecallCount
Month,
01,3
02,5
03,26
04,11
05,87
06,109
07,9
08,125
09,27
